In [29]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
from torch import nn
import torch.nn.functional as F

In [30]:
df=pd.read_csv('Prepared_Data/DataWithPadding.csv')

In [4]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
df.user_id.nunique()

3000

In [6]:
df.order_id.nunique()

189395

In [7]:
df.drop(columns=['user_id','order_number','order_id'],inplace=True)

In [8]:
#df.drop(columns=['order_dow','order_hour_of_day','days_since_prior_order'],inplace=True)

In [9]:
df = df.fillna(0)

In [10]:
df

,45,196,260,329,432,651,890,1025,1158,1194,1244,1463,1511,1940,1999,2078,2086,2228,2295,2314,2452,2825,2855,2966,3298,3376,3464,3599,3896,3952,3957,4086,4210,4421,4472,4562,4605,4656,4799,4920,...,45603,45633,46584,46616,46654,46667,46676,46720,46802,46820,46906,46969,46979,47042,47141,47144,47209,47626,47630,47672,47734,47759,47766,47977,48205,48364,48628,48679,48745,48775,49075,49175,49191,49235,49383,49520,49683,order_dow,order_hour_of_day,days_since_prior_order
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,13.0,5.0
299996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,3.0
299997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,16.0,3.0
299998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,19.0


## Model

### creating sequence

In [11]:
seq=20
batch=df.shape[0]//seq
X=[]
y=[]
n=0
m=n+1
for i in range(batch):
    X.append(df[n:(n+seq)])
    if((m+seq)<len(df)):
        y.append(df[m:(m+seq)])
    else:
        y.append(df[n:(n+seq)])
    n=n+20
    m=n+1
#y[-1] = y[-1].append(df.iloc[0])

In [12]:
X = np.array([np.array(i) for i in X])
y = np.array([np.array(i) for i in y])

In [13]:
users = X.shape[0]//5
features = X.shape[-1]

In [14]:
X = X.reshape(users,5,20,features)
y = y.reshape(users,5,20,features)

In [15]:
y.shape

(3000, 5, 20, 503)

In [16]:
y = y[:,:,:,:500]

In [17]:
y.shape

(3000, 5, 20, 500)

In [18]:
test_idx = int(len(X)*(1-0.2))
test_idx

2400

In [19]:
X_train, X_test = X[:test_idx], X[test_idx:]
y_train, y_test = y[:test_idx], y[test_idx:]

In [20]:
X_test.shape

(600, 5, 20, 503)

In [21]:
val_idx = int(len(X_train)*(1-0.25))
val_idx

1800

In [22]:
X_data, X_val = X_train[:val_idx], X_train[val_idx:]
y_data, y_val = y_train[:val_idx], y_train[val_idx:]

In [23]:
print(X_train.shape)
print(y_train.shape)

(2400, 5, 20, 503)
(2400, 5, 20, 500)


In [24]:
print(X_val.shape)
print(y_val.shape)

(600, 5, 20, 503)
(600, 5, 20, 500)


In [25]:
print(X_test.shape)
print(y_test.shape)

(600, 5, 20, 503)
(600, 5, 20, 500)


In [ ]:
np.save('X_test_without_pad', X_test)
np.save('y_test_without_pad', y_test)

In [26]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU')

Training on GPU!


In [62]:
class InstaCartLSTM(nn.Module):
    
    def __init__(self, Data, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.orders = Data
        self.features = Data.shape[-1]
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(self.features, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, 500)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm        
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [63]:
data = X_train.reshape(-1,X_train.shape[-1])
class_weight=[]
for i in range(data.shape[1]):
    positive=len(np.array(data[:,i][data[:,i]==1]))
    negative=len(np.array(data[:,i][data[:,i]==0]))
    class_weight.append(negative/positive)
class_weight = torch.as_tensor(class_weight, dtype=torch.float)

In [64]:
class_weight = class_weight[:500]

In [65]:
class_weight.shape

torch.Size([500])

In [66]:
def train(net, X_train,y_train,X_val,y_val, epochs=10, batch_size=5, seq_length=100, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weight.cuda())
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.orders)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        train_set=len(X_train)
        
        for i in range(train_set):
            counter += 1
                        
            inputs, targets = torch.from_numpy(X_train[i]), torch.from_numpy(y_train[i])            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])            
            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model            
            output, h = net(inputs.float(), h)                                
            loss = criterion(output, targets.reshape(batch_size*seq_length,-1))
            #print(loss)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                
                validation_set = len(X_val)
                for i in range(validation_set):                    
                    x, y = torch.from_numpy(X_val[i]), torch.from_numpy(y_val[i])
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs.float(), val_h)
                    val_loss = criterion(output,targets.reshape(batch_size*seq_length,-1))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [67]:
# define and print the net
n_hidden=1024
n_layers=1

net = InstaCartLSTM(X_train, n_hidden, n_layers)
net = net.float()
print(net)

InstaCartLSTM(
  (lstm): LSTM(503, 1024, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1024, out_features=500, bias=True)
)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [68]:
batch_size = 5
seq_length = 20
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, X_train,y_train,X_val,y_val,epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=2000)

Epoch: 1/50... Step: 2000... Loss: 0.8832... Val Loss: 0.9849
Epoch: 2/50... Step: 4000... Loss: 0.6409... Val Loss: 0.9064
Epoch: 3/50... Step: 6000... Loss: 0.3981... Val Loss: 0.8322
Epoch: 4/50... Step: 8000... Loss: 0.6821... Val Loss: 0.7662
Epoch: 5/50... Step: 10000... Loss: 0.5947... Val Loss: 0.7253
Epoch: 5/50... Step: 12000... Loss: 0.3217... Val Loss: 0.6535
Epoch: 6/50... Step: 14000... Loss: 0.4965... Val Loss: 0.6624
Epoch: 7/50... Step: 16000... Loss: 0.5558... Val Loss: 0.6662
Epoch: 8/50... Step: 18000... Loss: 0.2612... Val Loss: 0.6445
Epoch: 9/50... Step: 20000... Loss: 0.5310... Val Loss: 0.6083
Epoch: 10/50... Step: 22000... Loss: 0.5011... Val Loss: 0.5934
Epoch: 10/50... Step: 24000... Loss: 0.2396... Val Loss: 0.5223
Epoch: 11/50... Step: 26000... Loss: 0.4161... Val Loss: 0.5293
Epoch: 12/50... Step: 28000... Loss: 0.4770... Val Loss: 0.5532
Epoch: 13/50... Step: 30000... Loss: 0.2048... Val Loss: 0.5481
Epoch: 14/50... Step: 32000... Loss: 0.4390... Val Los

In [69]:
torch.save(net, 'Model/lstm_model_1024.net')